In [ ]:
import duckdb
from fsspec import filesystem
import os
cn = duckdb.connect()
cn.register_filesystem(filesystem('gcs'))

In [2]:
bucket = os.getenv('GCS_BUCKET')
gs_path = f"gs://{bucket}/test_data/data*.csv"
sql = f"select * from read_csv_auto('{gs_path}') limit 50"
cn.sql(sql).show(max_width = 1000)

┌────────────┬─────────────┬──────────────────┬──────────────────────┬──────────────┬───────────┬───────────┬────────────┬────────────┬──────────────────────────────────────┐
│  Latitude  │  Longitude  │ CreditCardNumber │        Email         │ PhoneNumber  │ FirstName │ LastName  │    Date    │  NetWorth  │                TxnKey                │
│   double   │   double    │      int64       │       varchar        │   varchar    │  varchar  │  varchar  │    date    │   double   │               varchar                │
├────────────┼─────────────┼──────────────────┼──────────────────────┼──────────────┼───────────┼───────────┼────────────┼────────────┼──────────────────────────────────────┤
│  63.305305 │ -154.303574 │ 6011517609545153 │ WMUVtRc@RfjZciH.com  │ 410-815-2637 │ Bertrand  │ Gutkowski │ 2018-05-23 │      10.16 │ 6b691a43-73d9-4784-9397-5a8317418b0f │
│ -48.165188 │  -37.616329 │ 6011504282599022 │ MxnNAYJ@ExcgPvo.ru   │ 927-108-4615 │ Ivah      │ Hoeger    │ 1980-04-29 │   

In [3]:
b_sql = f"""
select Date as rpt_dt 
    ,count(*) as row_cnt
    ,sum(NetWorth) as tot_net_worth
    ,count(distinct firstName) as fName_cnt
from read_csv_auto('{gs_path}')
group by all
"""
cn.sql(b_sql).show()

┌────────────┬─────────┬────────────────────┬───────────┐
│   rpt_dt   │ row_cnt │   tot_net_worth    │ fName_cnt │
│    date    │  int64  │       double       │   int64   │
├────────────┼─────────┼────────────────────┼───────────┤
│ 1986-05-27 │       4 │  6160931.680000001 │         4 │
│ 1977-10-23 │       6 │            14263.1 │         6 │
│ 2006-02-02 │       7 │           121700.7 │         7 │
│ 1979-09-24 │       6 │         1307989.06 │         6 │
│ 2011-09-21 │       3 │          611845.71 │         3 │
│ 1991-04-04 │      11 │         4839451.42 │        11 │
│ 2007-11-06 │       6 │ 3289598.9299999997 │         6 │
│ 1988-04-13 │       8 │ 15826.199999999997 │         8 │
│ 1995-09-23 │       7 │ 3227815.2199999997 │         7 │
│ 1982-10-07 │       3 │          362582.35 │         3 │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│ 1998-12-06 │

In [4]:
b_sql = f"""
select Date as rpt_dt 
    ,count(*) as row_cnt
    ,sum(NetWorth) as tot_net_worth
    ,count(distinct firstName) as fName_cnt
from read_csv_auto('{gs_path}')
group by all
"""

gs_write_path = f"gs://{bucket}/test_data/data_tsfm.parquet"
sql = f"""
COPY ({b_sql}) to '{gs_write_path}' (FORMAT PARQUET)
"""

cn.execute(sql)

In [5]:
sql = f"""
select *
from read_parquet('gs://{bucket}/test_data/data_tsfm.parquet')
"""
cn.sql(sql).show()

┌────────────┬─────────┬────────────────────┬───────────┐
│   rpt_dt   │ row_cnt │   tot_net_worth    │ fName_cnt │
│    date    │  int64  │       double       │   int64   │
├────────────┼─────────┼────────────────────┼───────────┤
│ 1972-03-27 │       3 │ 427365.48000000004 │         3 │
│ 2002-10-18 │       7 │          567661.53 │         7 │
│ 1980-11-03 │       5 │  7905259.649999999 │         5 │
│ 1995-01-14 │       5 │         1052403.82 │         5 │
│ 1975-03-06 │       7 │  8872927.620000001 │         7 │
│ 1974-01-19 │       9 │         5837204.09 │         9 │
│ 1970-05-20 │       7 │         2234565.54 │         7 │
│ 2019-04-19 │       1 │               2.03 │         1 │
│ 1995-09-16 │       6 │  82717.98000000001 │         6 │
│ 1984-11-25 │       5 │ 2491062.0100000002 │         5 │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│     ·      │       · │              ·     │         · │
│ 2016-10-12 │